In [1]:
pip install neo4j

     |████████████████████████████████| 176 kB 2.8 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for neo4j: filename=neo4j-5.7.0-py3-none-any.whl size=243940 sha256=32171ee3d01080136927dc798e6a075086a53e9e92c8e8f19bfe464d2a631634
  Stored in directory: /Users/chetan/Library/Caches/pip/wheels/6d/be/1e/749b84bba4b7429145ea09ccb46114fab4748625eb319f286a
Successfully built neo4j
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from neo4j import GraphDatabase
import logging
from neo4j.exceptions import ServiceUnavailable

class App:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        # Don't forget to close the driver connection when you are finished with it
        self.driver.close()

    def create_route(self, person1_name, person2_name):
        with self.driver.session(database="neo4j") as session:
            # Write transactions allow the driver to handle retries and transient errors
            result = session.execute_write(
                self._create_and_return_friendship, person1_name, person2_name)
            for row in result:
                print("Created Travel To relationship between: {p1}, {p2}".format(p1=row['p1'], p2=row['p2']))

    @staticmethod
    def _create_and_return_friendship(tx, person1_name, person2_name):
        # To learn more about the Cypher syntax, see https://neo4j.com/docs/cypher-manual/current/
        # The Reference Card is also a good resource for keywords https://neo4j.com/docs/cypher-refcard/current/
        query = (
            "CREATE (:Place {name: 'Robert F. Wagner Jr. Park', latitude: 40.7198, longitude: -74.0001, type: 'museum'})"
            "CREATE (:Place {name: 'The Skyscraper Museum', latitude: 40.7156, longitude: -74.0090, type: 'museum'}) "
            "CREATE (p1)-[:TRAVEL_TO]->(p2) "
            "RETURN p1, p2"
        )
        result = tx.run(query, person1_name=person1_name, person2_name=person2_name)
        try:
            return [{"p1": row["p1"]["name"], "p2": row["p2"]["name"]}
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    def find_person(self, person_name):
        with self.driver.session(database="neo4j") as session:
            result = session.execute_read(self._find_and_return_person, person_name)
            for row in result:
                print("Found : {row}".format(row=row))

    @staticmethod
    def _find_and_return_person(tx, person_name):
        query = (
            "MATCH (p:Place) "
            "WHERE p.name = $name "
            "RETURN p.name AS name"
        )
        result = tx.run(query, person_name=person_name)
        return [row["name"] for row in result]


if __name__ == "__main__":
    # Aura queries use an encrypted connection using the "neo4j+s" URI scheme
    uri = "neo4j+s://<Bolt url for Neo4j Aura instance>"
    user = "cnain"
    password = ""
    app = App(uri, user, password)
    app.create_friendship("", "David")
    app.find_person("Alice")
    app.close()